In [2]:
import sys
sys.path.append(r'D:\PyCharmProjects\VFPUMC02')
sys.path.append(r'C:\Users\Administrator\PycharmProjects\VFPUMC02')
sys.path.append(r'/root/VFPUMC02')
%load_ext autoreload
%autoreload 2

In [3]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import *
from consts.Constants import DATASETS_PATH
import os
from datasets.VFPU_GEN_DATASET import VFPU_GEN_DATASET
from enums.DataSetsName import DataSetsName
from ctgan import CTGAN
from classfiers.TwoStep import TwoStep
import pandas as pd
from vf4lr.VF_LR import VF_LR
from classfiers.VF_LinearRegression import VF_LinearRegression
from classfiers.VF_TwoStep import VF_TwoStep
from utils.FateUtils import convert_ipynb_to_py
from gen.VFPU_GEN import VFPU_GEN
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from utils.FateUtils import convert_ipynb_to_py

In [6]:
dataset = VFPU_GEN_DATASET()

In [10]:
ctgan = CTGAN(epochs=5)

clf =  LogisticRegression(solver='sag', max_iter=5)
reg =  Ridge(alpha=0.1, solver='sag', max_iter=5)
two_step = TwoStep(
    base_classifier=clf,
    base_regressor=reg,
    max_iter=5,
    k=0.25
)

vf_clf = VF_LR(learning_rate=0.2,epoch_num=5,batch_size=64)
vf_reg = VF_LinearRegression(config={'lr': 0.01,'lambda': 0.1,'n_iters':5})
vf_two_step = VF_TwoStep(
    clf=vf_clf,
    reg=vf_reg,
    k=0.25,
    max_iter=5
)

2025-01-24 00:33:19,395 - ConsoleLogger - INFO - TwoStep 类已初始化: max_iter=5, k=0.25, base_classifier=LogisticRegression, base_regressor=Ridge
2025-01-24 00:33:19,395 - INFO - TwoStep 类已初始化: max_iter=5, k=0.25, base_classifier=LogisticRegression, base_regressor=Ridge
2025-01-24 00:33:19,396 - ConsoleLogger - INFO - VF_LR 模型初始化完成。
2025-01-24 00:33:19,396 - INFO - VF_LR 模型初始化完成。
2025-01-24 00:33:19,397 - ConsoleLogger - INFO - 模型配置: {'learning_rate': 0.2, 'epoch_num': 5, 'batch_size': 64, 'client_num': 2}
2025-01-24 00:33:19,397 - INFO - 模型配置: {'learning_rate': 0.2, 'epoch_num': 5, 'batch_size': 64, 'client_num': 2}
2025-01-24 00:33:19,398 - ConsoleLogger - INFO - VF_LinearRegression 模型初始化完成。
2025-01-24 00:33:19,398 - INFO - VF_LinearRegression 模型初始化完成。
2025-01-24 00:33:19,399 - ConsoleLogger - INFO - 模型配置: {'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}
2025-01-24 00:33:19,399 - INFO - 模型配置: {'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}
2025-01-24 00:33:19,400 - INFO - VF_TwoStep 类已成功初始化: max_iter=

In [12]:
for dataset_enum in DataSetsName:
    print(dataset_enum.value)
    df_A, df_B, y = dataset.get_dataset_parts(dataset_enum.value)
    print(f"df_A 的形状: {df_A.shape}")
    print(f"df_B 的形状: {df_B.shape}")
    print(f"y 的形状: {y.shape}")
    miss_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for miss_rate in miss_rates:
        print(f"miss_rate的值为{miss_rate}")
        complete_df_B, incomplete_df_B, missing_df_B = constru_row_miss_df(df_B, miss_rate)
        print(f"complete_df_B 的形状: {complete_df_B.shape}")
        print(f"incomplete_df_B 的形状: {incomplete_df_B.shape}")
        print(f"missing_df_B 的形状: {missing_df_B.shape}")
        vfpu_gen = VFPU_GEN(two_step,vf_two_step,ctgan,df_A,incomplete_df_B)
        m = incomplete_df_B.shape[1]
        # 生成 k 列， 预测 m - k 列
        for k in range(m + 1):
            predict_cols = incomplete_df_B.columns[:m-k]
            vfpu_gen.fit(predict_cols)
            rmse, mse, mae, r2 = evaluate_imputed_data(missing_df_B, synthetic_data)
            print(f"rmse = {rmse}, mse = {mse}, mae = {mae}, r2 = {r2} ")

BANK
df_A 的形状: (41188, 31)
df_B 的形状: (41188, 32)
y 的形状: (41188, 1)
miss_rate的值为0.1
complete_df_B 的形状: (41188, 32)
incomplete_df_B 的形状: (37070, 32)
missing_df_B 的形状: (4118, 32)


KeyboardInterrupt: 

In [4]:
convert_ipynb_to_py('测试生成脚本.ipynb','testGen_2.py')

In [18]:
result = {}
for dataset_enum in DataSetsName:
    result[dataset_enum.value] = {}
    print(dataset_enum.value)
    df_A, df_B, y = dataset.get_dataset_parts(dataset_enum.value)
    print(f"df_A 的形状: {df_A.shape}")
    print(f"df_B 的形状: {df_B.shape}")
    print(f"y 的形状: {y.shape}")
    miss_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for miss_rate in miss_rates:
        result[dataset_enum.value][miss_rate] = {}
        print(f"miss_rate的值为{miss_rate}")
        complete_df_B, incomplete_df_B, missing_df_B = constru_row_miss_df(df_B, miss_rate)
        print(f"complete_df_B 的形状: {complete_df_B.shape}")
        print(f"incomplete_df_B 的形状: {incomplete_df_B.shape}")
        print(f"missing_df_B 的形状: {missing_df_B.shape}")
        # vfpu_gen = VFPU_GEN(two_step,vf_two_step,ctgan,df_A,incomplete_df_B)
        m = incomplete_df_B.shape[1]
        # 生成 k 列， 预测 m - k 列
        for k in range(m + 1):
            predict_cols = incomplete_df_B.columns[:m-k]
            # vfpu_gen.fit(predict_cols)
            # synthetic_data = vfpu_gen.get_synthetic_data()
            # rmse, mse, mae, r2 = evaluate_imputed_data(missing_df_B, synthetic_data)
            rmse, mse, mae, r2 = dataset_enum.value, miss_rate, k, 0
            print(f"rmse = {rmse}, mse = {mse}, mae = {mae}, r2 = {r2} ")
            result[dataset_enum.value][miss_rate][k] = [rmse, mse, mae, r2]

BANK
df_A 的形状: (41188, 31)
df_B 的形状: (41188, 32)
y 的形状: (41188, 1)
miss_rate的值为0.1
complete_df_B 的形状: (41188, 32)
incomplete_df_B 的形状: (37070, 32)
missing_df_B 的形状: (4118, 32)
rmse = BANK, mse = 0.1, mae = 0, r2 = 0 
rmse = BANK, mse = 0.1, mae = 1, r2 = 0 
rmse = BANK, mse = 0.1, mae = 2, r2 = 0 
rmse = BANK, mse = 0.1, mae = 3, r2 = 0 
rmse = BANK, mse = 0.1, mae = 4, r2 = 0 
rmse = BANK, mse = 0.1, mae = 5, r2 = 0 
rmse = BANK, mse = 0.1, mae = 6, r2 = 0 
rmse = BANK, mse = 0.1, mae = 7, r2 = 0 
rmse = BANK, mse = 0.1, mae = 8, r2 = 0 
rmse = BANK, mse = 0.1, mae = 9, r2 = 0 
rmse = BANK, mse = 0.1, mae = 10, r2 = 0 
rmse = BANK, mse = 0.1, mae = 11, r2 = 0 
rmse = BANK, mse = 0.1, mae = 12, r2 = 0 
rmse = BANK, mse = 0.1, mae = 13, r2 = 0 
rmse = BANK, mse = 0.1, mae = 14, r2 = 0 
rmse = BANK, mse = 0.1, mae = 15, r2 = 0 
rmse = BANK, mse = 0.1, mae = 16, r2 = 0 
rmse = BANK, mse = 0.1, mae = 17, r2 = 0 
rmse = BANK, mse = 0.1, mae = 18, r2 = 0 
rmse = BANK, mse = 0.1, mae = 19, r2

In [19]:
result

{'BANK': {0.1: {0: ('BANK', 0.1, 0, 0),
   1: ('BANK', 0.1, 1, 0),
   2: ('BANK', 0.1, 2, 0),
   3: ('BANK', 0.1, 3, 0),
   4: ('BANK', 0.1, 4, 0),
   5: ('BANK', 0.1, 5, 0),
   6: ('BANK', 0.1, 6, 0),
   7: ('BANK', 0.1, 7, 0),
   8: ('BANK', 0.1, 8, 0),
   9: ('BANK', 0.1, 9, 0),
   10: ('BANK', 0.1, 10, 0),
   11: ('BANK', 0.1, 11, 0),
   12: ('BANK', 0.1, 12, 0),
   13: ('BANK', 0.1, 13, 0),
   14: ('BANK', 0.1, 14, 0),
   15: ('BANK', 0.1, 15, 0),
   16: ('BANK', 0.1, 16, 0),
   17: ('BANK', 0.1, 17, 0),
   18: ('BANK', 0.1, 18, 0),
   19: ('BANK', 0.1, 19, 0),
   20: ('BANK', 0.1, 20, 0),
   21: ('BANK', 0.1, 21, 0),
   22: ('BANK', 0.1, 22, 0),
   23: ('BANK', 0.1, 23, 0),
   24: ('BANK', 0.1, 24, 0),
   25: ('BANK', 0.1, 25, 0),
   26: ('BANK', 0.1, 26, 0),
   27: ('BANK', 0.1, 27, 0),
   28: ('BANK', 0.1, 28, 0),
   29: ('BANK', 0.1, 29, 0),
   30: ('BANK', 0.1, 30, 0),
   31: ('BANK', 0.1, 31, 0),
   32: ('BANK', 0.1, 32, 0)},
  0.2: {0: ('BANK', 0.2, 0, 0),
   1: ('BANK', 0.